# Лабораторная работа №5.
## Малютин Александр БПМ152.
### Вариант 20.

## Задача 5.1. 
Дана  система уравнений  $Ax=b$. Найти решение  системы  с помощью метода Гаусса. Выполнить 10 итераций по методу Зейделя. Принимая решение,  полученное с помощью метода Гаусса  за точное, найти величину абсолютной погрешности итерационного решения.

ПОРЯДОК  РЕШЕНИЯ  ЗАДАЧИ:
1. Задать матрицу системы $A$ и вектор правой части $b$. Используя встроенную функцию  $lsolve$ пакета $MATHCAD$, найти решение системы $Ax=b$ с помощью метода Гаусса. 
2. Преобразовать систему $Ax=b$ к виду $x=Bx+c$, удобному для итераций. Проверить выполнение достаточного условия сходимости итерационных методов $||B||_{\infty}<1$.
3. Используя функцию $zeid$  (см. ПРИЛОЖЕНИЕ 5.B), выполнить 10  итераций по методу Зейделя; взять любое начальное приближение. Принимая решение, полученное в п. 1 за точное, найти величину абсолютной погрешности итерационного решения (использовать норму $||\cdot ||_\infty$). 
4. Взять другое начальное приближение. Объяснить полученные результаты.

## Задача  5.2.  
Для системы уравнений $Ax=b$ из задачи 5.1 найти  решение  по методу Зейделя с  точностью $\epsilon=10^{-6}$ , взяв любое начальное приближение. Для этого  модифицировать функцию zeid  так, чтобы  решение вычислялось с заданной точностью $\epsilon$. Предусмотреть подсчет  количества итераций, потребовавшихся для достижения точности $\epsilon$. 


In [1]:
import numpy as np
from numpy import sqrt
from numpy.linalg import norm

### Дано:

In [2]:
epsilon = 10**(-6)

A = np.array(
    [[39.6,   0, 17.5,   9.9, 12], 
     [79.2, 120,    0,  39.6,  0],
     [19.8, -21,   46,     0,  5],
     [49.5,  19,   19,  89.1,  0],
     [9.9 ,  25,   10, -39.6, 85]])

b = np.array([-34.35, -530, 102.1, -286.5, 101.3]).reshape(-1,1)

n = len(A)

### Решение встроенным методом:

In [3]:
ExactX = np.linalg.solve(A,b)
print(ExactX)

[[-1.19191919]
 [-2.9       ]
 [ 1.3       ]
 [-2.21212121]
 [ 1.        ]]


### Определим метод Гауса

In [4]:
def GaussMethod(matrix, right):
    A = np.copy(matrix)
    b = np.copy(right)
    n = len(A)
    
    if (n != len(b)) and (A.shape[0] != A.shape[1]):
        print("ERROR!!!!")
        return 0    
    
    #Forward
    for i in range(n - 1):
        for j in range(i + 1, n):
            mu = A[j, i]/A[i, i]
            A[j] -= A[i]*mu
            b[j] -= b[i]*mu
    
    #Backward
    for i in range(n - 1, 0, -1):
        for j in range(i - 1, -1, -1):
            mu = A[j, i]/A[i, i]
            A[j] -= A[i]*mu
            b[j] -= b[i]*mu

    return np.array([b[i]/A[i, i] for i in range(n)])

In [5]:
GaussX = GaussMethod(A,b)
print(GaussX)

[[-1.19191919]
 [-2.9       ]
 [ 1.3       ]
 [-2.21212121]
 [ 1.        ]]


### Приведём систему к виду $x = Bx + c$. 
#### Построим матрицу $B$ и вектор $c$:


In [6]:
def PB(A):
    n = len(A)
    B = np.array([[-A[i,j]/A[i,i] for i in range(n)] for j in range(n)])
    B += np.eye(n);
    return B

def Pc(A, b):
    n = len(A)
    return np.array([b[i]/A[i,i] for i in range(n)])     

In [7]:
B = PB(A)

c = Pc(A, b)

print(B)
print(c)

[[ 0.         -0.66       -0.43043478 -0.55555556 -0.11647059]
 [ 0.          0.          0.45652174 -0.21324355 -0.29411765]
 [-0.44191919  0.          0.         -0.21324355 -0.11764706]
 [-0.25       -0.33        0.          0.          0.46588235]
 [-0.3030303   0.         -0.10869565  0.          0.        ]]
[[-0.86742424]
 [-4.41666667]
 [ 2.21956522]
 [-3.21548822]
 [ 1.19176471]]


#### Проверим достаточные условие сходимости итерационых методов $||B||_{\infty}<1$.

In [8]:
normB = norm(B, ord=np.inf) 
print("||B|| =", normB )

||B|| = 1.7624609263995454


### Определим метод Зейделя:

In [9]:
def SeidelMethod(B, c, x0, k=-1, eps=-1):
    n = len(B)
    y = np.copy(x0)
    
    U = np.triu(B)
    L = np.tril(B)
    
    m = 0
    stopCondition = True
    while stopCondition:
        m += 1
        x = np.copy(y)
            
        u = np.sum(U[i] * x[i] for i in range(n)).reshape(-1, 1)
        l = np.sum(L[i] * x[i] for i in range(n)).reshape(-1, 1)
        
        y = u + l + c
        
        if (k > 0):
            stopCondition = m < k
            
        if (eps > 0):
            stopCondition = np.linalg.norm(y - x, ord=2) >= eps
        
            
    print("steps =", m)
    return y.reshape(-1,1)

In [10]:
Seidel10X = SeidelMethod(B, c, np.zeros(n), k=10)
print(Seidel10X)
np.linalg.norm(ExactX - Seidel10X, ord=2)

steps = 10
[[-1.18131115]
 [-2.8860127 ]
 [ 1.29602888]
 [-2.19730322]
 [ 1.001576  ]]


0.02336667829171065

In [11]:
SeidelX = SeidelMethod(B, c, np.zeros(n), eps=epsilon)
print(SeidelX)
np.linalg.norm(ExactX - SeidelX, ord=2)

steps = 31
[[-1.19191935]
 [-2.90000009]
 [ 1.29999988]
 [-2.21212134]
 [ 0.99999997]]


2.540042993856786e-07

In [12]:
SeidelX = SeidelMethod(B, c, np.zeros(n) + 1, eps=epsilon)
print(SeidelX)
np.linalg.norm(ExactX - SeidelX, ord=2)

steps = 31
[[-1.19191929]
 [-2.89999986]
 [ 1.29999977]
 [-2.21212116]
 [ 0.99999995]]


2.9728933860802303e-07

### Ответы совпали с указанной точностью. Ура!